# MIST / TSST Examples

In [ ]:
from pathlib import Path

import re

import pandas as pd
import numpy as np

import biopsykit as bp
from biopsykit.protocols import MIST, TSST

import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib widget
%load_ext autoreload
%autoreload 2

In [ ]:
sns.set(style='ticks')

**Note:**

This example illustrates how to process and plot data for the Montreal Imaging Stress Task (MIST) and the Trier Social Stress Task (TSST). For this, it uses functions to split data, compute parameter, etc. that are part of the respective classes in `BioPsyKit` representing the different stress protocols (e.g., `bp.protocols.MIST()` or `bp.protocols.TSST()`). These functions simplify the computation for these protocols. If you developed your own stress procotol and want to use a similar processing approach, `BioPsyKit` also offers these functions to be used stand-alone and without having to create a new object for your stress protocol.

You can find the equivalent functions here:

* `bp.protocols.MIST.split_subphases(data, is_group_dict)` $\rightarrow$ `bp.utils.data_processing.split_subphases(data, subphase_names, subphase_times, is_group_dict)` 
* `bp.protocols.MIST.split_groups(data, dict_condition)` $\rightarrow$ `bp.utils.data_processing.split_groups(data, dict_condition)`
* `bp.protocols.MIST.hr_mean_se_subphases(data, is_group_dict)` $\rightarrow$ `bp.utils.data_processing.mean_se_nested_dict(data, subphases, is_group_dict)`
* `bp.protocols.MIST.hr_mean_plot(data)` $\rightarrow$ `bp.protocols.plotting.hr_mean_plot(data)` ($\rightarrow$ see `ECG_Analysis_Example.ipynb` for examples)
* `bp.protocols.MIST.saliva_plot(data, biomarker)` $\rightarrow$ `bp.protocols.plotting.saliva_plot(data, biomarker, saliva_times, test_times)` ($\rightarrow$ see `Saliva_Example.ipynb` for an example)

If your protocol does **not** have subphases (only phases, like the TSST for example only has the phases *Preparation*, *Speaking*, *Mental Arithmetic*) you can ue all these functions equivalently, just without splitting your data into subphases!

## MIST

Create a MIST object with default parameter:
* *Phases*: MIST1, MIST2, MIST3
* *Subphases*: BL, AT, FB
* *Subphase Durations*: 60, 240, 0 (Feedback Interval has length 0 because it's of variable length for each MIST phase and will later be inferred from the data)

If you want to create a MIST object with other parameters, you can pass this to the constructor `MIST()`

In [ ]:
mist = MIST()
mist

### ECG Results

#### Load Data

**Subject Conditions**

In [ ]:
dict_condition = bp.io.load_subject_condition_list("../example_data/condition_list.csv")

**HR Phase Dict**:

`dict_phase`: Dictionary with Heart Rate data per MIST Phase
* keys = Phase names
* values = Heart Rate data (pandas DataFrame)

In [ ]:
# Load all sheets of the Excel file containing normalized heart rate date for all subjects, split into the different phases
dict_phase = pd.read_excel("../example_data/hr_phase_export_sample_normalized.xlsx", sheet_name=None, index_col="Time")

#### Rearrange Data

**Split 'Phase dict' into 'Subphase dict'**:

`dict_subph`: Nested dictionary with heart rate data per MIST Phase and Subphase, respectively
* keys = Phase names
* values = dictionary with Heart Rate data per Subphase:
    * keys = Subphase names
    * values = Heart Rate data (pandas DataFrame)

In [ ]:
dict_subph = mist.split_subphases(dict_phase)

**Split HR Phase Dict into Conditions**

In [ ]:
dict_groups = mist.split_groups(dict_phase, dict_condition)

**Split 'Phase dict' of each condition into 'Subphase dict'**

In [ ]:
dict_subph_groups = mist.split_subphases(dict_groups, is_group_dict=True)

#### Compute Parameter

**Mean and Standard Error of Heart Rate during each Subphase** (for all data and split into conditions)

In [ ]:
bp.utils.data_processing.mean_per_subject_nested_dict(dict_phase, param_name="HR")

In [ ]:
# compute 'mse dataframe' for all data
mean_se = mist.hr_mean_se_subphases(dict_subph)
mean_se

In [ ]:
# compute 'mse dataframe' for each condition individually
mean_se_groups = mist.hr_mean_se_subphases(dict_subph_groups, is_group_dict=True)
mean_se_groups

#### Plots

##### HR Ensemble Plot

In [ ]:
fig, ax = plt.subplots(figsize=(10,5))
mist.hr_ensemble_plot(dict_phase, ax=ax)

##### HR Mean and Standard Error per MIST Phase

In [ ]:
fig, ax = plt.subplots(figsize=(10, 5))
ax = mist.hr_mean_plot(data=mean_se, ax=ax)

In [ ]:
fig, ax = plt.subplots(figsize=(10, 5))
ax = mist.hr_mean_plot(data=mean_se_groups, ax=ax)

### Saliva Results

In [ ]:
saliva_mist = bp.example_data.get_saliva_example(saliva_times=[-30, -1, 30, 40, 50, 60, 70])
display(saliva_mist)

In [ ]:
saliva_mist_mean = bp.saliva.utils.mean_se(saliva_mist, biomarker_type='cortisol')
saliva_mist_mean

In [ ]:
fig, ax = mist.saliva_plot(saliva_mist_mean, biomarker="cortisol", figsize=(10,5))
fig.tight_layout()

## TSST

In [ ]:
tsst = TSST()

In [ ]:
saliva_tsst_mean = bp.example_data.get_saliva_mean_se_example()

In [ ]:
fig, ax = tsst.saliva_plot(saliva_tsst_mean, biomarker="cortisol", figsize=(10, 5))
tsst.saliva_plot(saliva_tsst_mean, biomarker="amylase", ax=ax)
tsst.saliva_plot_combine_legend(fig, ax, biomarkers=['cortisol', 'amylase'], separate_legends=False)
fig.tight_layout()

In [ ]:
fig, ax = tsst.saliva_plot(saliva_tsst_mean, biomarker="il6", figsize=(10, 5))
fig.tight_layout()